In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
from pandas import DataFrame, Series

In [2]:
Base_Aloe_Ferox_Dataset = pd.ExcelFile('BaseSet.xlsx')

In [3]:
Base_Set = {}
for sheet in Base_Aloe_Ferox_Dataset.sheet_names:
    df = Base_Aloe_Ferox_Dataset.parse(sheet)
    Base_Set[sheet] = df

In [4]:
flowers_base = Base_Set["FLOWERS"]
buds_base = Base_Set["BUDS"]
fruit_base = Base_Set["FRUIT"]
No_Evidence_Base = Base_Set["No Evidence"]

In [5]:
flowers_base.head()

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,user_name,created_at,updated_at,...,species_guess,scientific_name,common_name,iconic_taxon_name,taxon_id,taxon_family_name,taxon_genus_name,taxon_species_name,taxon_subspecies_name,field:phenology (foa)
0,9865839,2017-06-24 3:22:22 PM SAST,2017-06-24,2017-06-24 13:22:22 UTC,Africa/Johannesburg,764003,gabriel_15,NaN,2018-02-15 07:32:45 UTC,2018-06-07 16:08:59 UTC,...,Cape aloe,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN
1,10787160,2011-07-15 00:00:00,2011-07-15,NaN,Africa/Johannesburg,660426,outramps-tanniedi,Di Turner,2011-07-18 12:28:39 UTC,2018-06-07 16:14:22 UTC,...,Cape aloe,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN
2,10787636,2011-07-29 00:00:00,2011-07-29,NaN,Africa/Johannesburg,660426,outramps-tanniedi,Di Turner,2011-08-02 01:09:32 UTC,2018-06-07 16:14:07 UTC,...,Cape aloe,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN
3,10790153,2011-08-14 00:00:00,2011-08-14,NaN,Africa/Johannesburg,7682,nicky,Nicola van Berkel,2011-09-01 07:26:07 UTC,2018-06-07 16:13:47 UTC,...,Cape aloe,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN
4,10799481,2012-05-26 00:00:00,2012-05-26,NaN,Africa/Johannesburg,660426,outramps-tanniedi,Di Turner,2012-06-15 01:15:18 UTC,2018-08-26 12:02:52 UTC,...,Cape aloe,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN


In [6]:
buds_base.head()

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,user_name,created_at,updated_at,...,scientific_name,common_name,iconic_taxon_name,taxon_id,taxon_family_name,taxon_genus_name,taxon_species_name,taxon_subspecies_name,taxon_variety_name,field:phenology (foa)
0,8154638,2017/04/30 7:56 AM EDT,2017-04-30,2017-04-30 11:56:00 UTC,Eastern Time (US & Canada),520,johnnybirder,Johnny Wilson,2017-09-28 16:43:38 UTC,2018-06-07 16:12:02 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
1,9272972,2007/07/01 9:41 AM BST,2007-07-01,2007-07-01 08:41:00 UTC,Europe/London,493784,mark185,Mark Wright,2017-12-27 13:16:33 UTC,2018-06-07 16:09:12 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
2,9865839,2017-06-24 3:22:22 PM SAST,2017-06-24,2017-06-24 13:22:22 UTC,Africa/Johannesburg,764003,gabriel_15,NaN,2018-02-15 07:32:45 UTC,2018-06-07 16:08:59 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
3,10787160,2011-07-15 00:00:00,2011-07-15,NaN,Africa/Johannesburg,660426,outramps-tanniedi,Di Turner,2011-07-18 12:28:39 UTC,2018-06-07 16:14:22 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
4,10787636,2011-07-29 00:00:00,2011-07-29,NaN,Africa/Johannesburg,660426,outramps-tanniedi,Di Turner,2011-08-02 01:09:32 UTC,2018-06-07 16:14:07 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN


In [7]:
fruit_base.head()

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,user_name,created_at,updated_at,...,scientific_name,common_name,iconic_taxon_name,taxon_id,taxon_family_name,taxon_genus_name,taxon_species_name,taxon_subspecies_name,taxon_variety_name,field:phenology (foa)
0,9034202,2017/11/24 2:54 PM SAST,2017-11-24,2017-11-24 12:54:00 UTC,Africa/Johannesburg,389997,shauns,Shaun Swanepoel,2017-12-03 21:28:02 UTC,2018-08-26 11:58:10 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
1,10832079,2012-10-12 00:00:00,2012-10-12,NaN,Africa/Johannesburg,7682,nicky,Nicola van Berkel,2012-11-30 08:16:37 UTC,2018-11-19 09:09:18 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
2,10871050,2013-07-17 00:00:00,2013-07-17,NaN,Africa/Johannesburg,660426,outramps-tanniedi,Di Turner,2013-07-30 01:59:08 UTC,2018-11-19 09:09:51 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
3,10875477,2013-09-25 00:00:00,2013-09-25,NaN,Africa/Johannesburg,7682,nicky,Nicola van Berkel,2013-10-17 17:28:07 UTC,2018-11-19 09:08:12 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
4,10962203,2013-09-12 00:00:00,2013-09-12,NaN,Africa/Johannesburg,383144,tonyrebelo,Tony Rebelo,2014-05-03 23:49:34 UTC,2023-10-13 09:04:15 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN


In [8]:
No_Evidence_Base.head()

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,user_name,created_at,updated_at,...,scientific_name,common_name,iconic_taxon_name,taxon_id,taxon_family_name,taxon_genus_name,taxon_species_name,taxon_subspecies_name,taxon_variety_name,field:phenology (foa)
0,11111885,2015-05-06 00:00:00,2015-05-06,NaN,Africa/Johannesburg,660426,outramps-tanniedi,Di Turner,2015-05-17 11:20:21 UTC,2023-05-26 11:54:43 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
1,11197854,2015-11-20 00:00:00,2015-11-20,NaN,Pretoria,711873,benjamin_walton,Benjamin Walton,2016-01-27 11:54:52 UTC,2024-05-27 09:40:27 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
2,11245126,2017-01-17 00:00:00,2017-01-17,NaN,Africa/Johannesburg,664422,lizegreenieza,Lize van der Merwe,2017-06-12 15:59:45 UTC,2023-09-06 10:04:38 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
3,20173194,2017-06-24 00:00:00,2017-06-24,NaN,Pretoria,389997,shauns,Shaun Swanepoel,2017-08-16 08:05:05 UTC,2023-09-06 10:04:34 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN
4,27389400,2015/01/07 4:23 AM PST,2015-01-07,2015-01-07 12:23:00 UTC,Pacific Time (US & Canada),1831095,douglasriverside,Douglas J. Long,2019-06-21 18:19:54 UTC,2020-10-11 11:04:47 UTC,...,Aloe ferox,Cape Aloe,Plantae,124412,Asphodelaceae,Aloe,Aloe ferox,NaN,NaN,NaN


In [9]:
missing_values = (
    flowers_base.isnull().sum()/len(flowers_base)*100
).astype(int)

print(f'Column\t\t\t% missing')
print(f'{"-"}'*35)
missing_values

Column			% missing
-----------------------------------


id                                    0
observed_on_string                    0
observed_on                           0
time_observed_at                      7
time_zone                             0
user_id                               0
user_login                            0
user_name                             4
created_at                            0
updated_at                            0
quality_grade                         0
license                               5
url                                   0
image_url                             0
sound_url                           100
tag_list                             93
description                          88
num_identification_agreements         0
num_identification_disagreements      0
captive_cultivated                    0
oauth_application_id                 88
place_guess                           0
latitude                              0
longitude                             0
positional_accuracy                  10


In [10]:
#all null columns or columns with 100% the same value have been removed
useful = ['id','time_observed_at','time_zone','image_url','num_identification_agreements','latitude','longitude','positional_accuracy','public_positional_accuracy','field:phenology (foa)']

In [11]:
flowers_one= flowers_base[useful]
buds_one = buds_base[useful]
fruits_one = fruit_base[useful]
No_Evidence_one = No_Evidence_Base[useful]

In [12]:
def missing_values(df):
    print(f'Column\t\t\t% missing')
    print(f'{"-"}'*35) 
    return (df.isnull().sum()/len(df)*100
    ).astype(int)

In [13]:
missing_values(flowers_one)

Column			% missing
-----------------------------------


id                                0
time_observed_at                  7
time_zone                         0
image_url                         0
num_identification_agreements     0
latitude                          0
longitude                         0
positional_accuracy              10
public_positional_accuracy       10
field:phenology (foa)            65
dtype: int64

In [14]:
missing_values(buds_one)

Column			% missing
-----------------------------------


id                                0
time_observed_at                  5
time_zone                         0
image_url                         0
num_identification_agreements     0
latitude                          0
longitude                         0
positional_accuracy              16
public_positional_accuracy       16
field:phenology (foa)            65
dtype: int64

In [15]:
missing_values(fruits_one)

Column			% missing
-----------------------------------


id                                0
time_observed_at                  2
time_zone                         0
image_url                         0
num_identification_agreements     0
latitude                          0
longitude                         0
positional_accuracy              19
public_positional_accuracy       19
field:phenology (foa)            64
dtype: int64

In [227]:
missing_values(No_Evidence_one)

Column			% missing
-----------------------------------


id                                0
time_observed_at                  1
time_zone                         0
image_url                         0
num_identification_agreements     0
latitude                          0
longitude                         0
positional_accuracy              15
public_positional_accuracy       15
field:phenology (foa)            99
dtype: int64

In [16]:
No_Evidence_one.sort_values(by='num_identification_agreements')

,id,time_observed_at,time_zone,image_url,num_identification_agreements,latitude,longitude,positional_accuracy,public_positional_accuracy,field:phenology (foa)
39,37608167,2020-01-10 13:27:00 UTC,Africa/Johannesburg,https://inaturalist-open-data.s3.amazonaws.com...,0,-32.535222,25.855982,10.0,10.0,NaN
40,37608181,2020-01-10 13:35:00 UTC,Africa/Johannesburg,https://inaturalist-open-data.s3.amazonaws.com...,0,-32.613195,25.895712,10.0,10.0,NaN
44,37675514,2020-01-16 07:38:00 UTC,Africa/Johannesburg,https://inaturalist-open-data.s3.amazonaws.com...,0,-33.100468,25.873825,10.0,10.0,NaN
52,37675584,2020-01-16 08:33:00 UTC,Africa/Johannesburg,https://inaturalist-open-data.s3.amazonaws.com...,0,-32.787900,25.490083,10.0,10.0,NaN
55,37675613,2020-01-16 08:53:00 UTC,Africa/Johannesburg,https://inaturalist-open-data.s3.amazonaws.com...,0,-32.874925,25.406588,10.0,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...
161,61032337,2020-09-25 09:54:00 UTC,Africa/Johannesburg,https://inaturalist-open-data.s3.amazonaws.com...,5,-33.707860,22.472620,4.0,4.0,NaN
164,61424795,2020-09-27 09:41:00 UTC,Africa/Johannesburg,https://inaturalist-open-data.s3.amazonaws.com...,5,-34.174528,22.133603,15.0,15.0,NaN
124,44175602,2020-04-28 15:27:00 UTC,Pretoria,https://static.inaturalist.org/photos/70038281...,5,-33.475339,21.731988,977.0,977.0,NaN
1,11197854,NaN,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,5,-34.019470,21.779820,1.0,1.0,NaN


#Idea to remove all rows not containing a value in phenology column

In [17]:
flowers_one.rename(columns={'field:phenology (foa)':'phenology'}, inplace=True)
buds_one.rename(columns={'field:phenology (foa)':'phenology'}, inplace=True)
fruits_one.rename(columns={'field:phenology (foa)':'phenology'}, inplace=True)
No_Evidence_one.rename(columns={'field:phenology (foa)':'phenology'}, inplace=True)

C:\Users\mckyl\AppData\Local\Temp\ipykernel_38908\2700043157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flowers_one.rename(columns={'field:phenology (foa)':'phenology'}, inplace=True)
C:\Users\mckyl\AppData\Local\Temp\ipykernel_38908\2700043157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buds_one.rename(columns={'field:phenology (foa)':'phenology'}, inplace=True)
C:\Users\mckyl\AppData\Local\Temp\ipykernel_38908\2700043157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [18]:
flowers_two = flowers_one[flowers_one.phenology.notnull()]
buds_two = buds_one[buds_one.phenology.notnull()]
fruits_two = fruits_one[fruits_one.phenology.notnull()]
No_Evidence_two = No_Evidence_one[No_Evidence_one.phenology.notnull()]


In [19]:
flowers_two.sort_values(by='phenology')

,id,time_observed_at,time_zone,image_url,num_identification_agreements,latitude,longitude,positional_accuracy,public_positional_accuracy,phenology
344,164929428,2023-05-27 06:21:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.725753,26.583278,5.0,5.0,Buds
342,163770902,2023-05-25 09:30:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.296353,26.462834,10.0,10.0,Buds
428,171992195,2023-06-24 09:40:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.048952,27.214480,10.0,10.0,Buds & Flowers
429,171992201,2023-06-24 09:40:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.048433,27.215142,10.0,10.0,Buds & Flowers
430,171992206,2023-06-24 09:40:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.048186,27.215688,10.0,10.0,Buds & Flowers
...,...,...,...,...,...,...,...,...,...,...
497,181403600,2023-08-27 10:14:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.503703,21.597907,10.0,10.0,Flowers & Fruit
512,181449836,2023-08-27 12:12:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,2,-33.547413,22.767891,10.0,10.0,Flowers & Fruit
486,181390794,2023-08-27 09:49:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,2,-33.511299,21.365059,10.0,10.0,Flowers & Fruit
485,181390774,2023-08-27 09:49:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,2,-33.518689,21.356090,10.0,10.0,Flowers & Fruit


In [232]:
buds_two.sort_values(by='phenology')

,id,time_observed_at,time_zone,image_url,num_identification_agreements,latitude,longitude,positional_accuracy,public_positional_accuracy,phenology
410,120189366,2022-06-04 12:37:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.494140,26.199388,20.0,20.0,Buds
590,171513336,2023-06-24 09:17:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.238346,27.001081,20.0,20.0,Buds
589,171513327,2023-06-24 09:15:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.249935,26.968641,20.0,20.0,Buds
587,171513321,2023-06-24 09:10:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.281131,26.891480,20.0,20.0,Buds
586,171513318,2023-06-24 09:10:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.281061,26.888890,20.0,20.0,Buds
...,...,...,...,...,...,...,...,...,...,...
757,181530442,2023-08-27 12:23:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.516882,22.957488,10.0,10.0,"Flower, Buds & Fruit"
758,181530443,2023-08-27 12:24:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.515560,22.960366,10.0,10.0,"Flower, Buds & Fruit"
734,181449571,2023-08-27 11:58:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,2,-33.485552,22.562042,10.0,10.0,"Flower, Buds & Fruit"
725,181390820,2023-08-27 09:51:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.501905,21.394802,10.0,10.0,"Flower, Buds & Fruit"


In [233]:
fruits_two.sort_values(by='phenology')

,id,time_observed_at,time_zone,image_url,num_identification_agreements,latitude,longitude,positional_accuracy,public_positional_accuracy,phenology
519,181449816,2023-08-27 12:11:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.549530,22.749781,10.0,10.0,Buds & Flowers
542,181530535,2023-08-27 12:31:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.474480,23.079894,10.0,10.0,"Flower, Buds & Fruit"
541,181530534,2023-08-27 12:31:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.474455,23.076702,10.0,10.0,"Flower, Buds & Fruit"
540,181530459,2023-08-27 12:25:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.506695,22.979609,10.0,10.0,"Flower, Buds & Fruit"
539,181530457,2023-08-27 12:25:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.508632,22.975375,10.0,10.0,"Flower, Buds & Fruit"
...,...,...,...,...,...,...,...,...,...,...
426,171191869,2023-07-04 10:45:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-32.491384,27.993007,10.0,10.0,Fruit
427,171191892,2023-07-04 10:47:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-32.499903,27.984548,10.0,10.0,Fruit
440,180732829,2023-08-27 15:18:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,2,-33.372756,25.030341,10.0,10.0,Fruit
402,134433490,2022-08-28 11:34:00 UTC,Pretoria,https://inaturalist-open-data.s3.amazonaws.com...,1,-33.748781,22.973049,20.0,20.0,Fruit
